In [1]:
import torch
import torch.nn as nn

In [14]:
a = torch.rand((2,3))
b,c = a.max(dim=-1)
a,b,c

(tensor([[0.4611, 0.4840, 0.5850],
         [0.7357, 0.5802, 0.6525]]),
 tensor([0.5850, 0.7357]),
 tensor([2, 0]))

In [8]:
a = torch.rand((2,1,3))
lstm = nn.LSTM(input_size=3, hidden_size=5)
output,(ht,ct) = lstm(a)
output.shape

torch.Size([2, 1, 5])

In [22]:
a = torch.arange(0,10,1)
a = torch.tensor([[1],[2],[3],[4],[5]])
b = torch.zeros((5,5))
b+a

In [7]:
a = torch.tensor([[1.,2.,3.]])
torch.logsumexp(a,dim=1)

tensor([3.4076])

In [75]:
torch.manual_seed(1)

tagset_size=10
batch_size = 5
START_TAG=0
STOP_TAG=9
tag2idx = {k:k for k in range(10)}
transitions = torch.rand((10,10))


feats = torch.rand((batch_size,10,10))
# Do the forward algorithm to compute the partition function
init_alphas = torch.full((batch_size, 1, tagset_size), -10000.)
# START_TAG has all of the score.
init_alphas[:,0,tag2idx[START_TAG]] = 0.

# Wrap in a variable so that we will get automatic backprop
forward_var = init_alphas

# Iterate through the sentence
for i in range(feats.shape[1]):
    feat = feats[:,i,:]
    emit_score = feat.view(batch_size, tagset_size, 1)
    next_tag_var = forward_var + transitions + emit_score
    forward_var = torch.logsumexp(next_tag_var,dim=-1).view(batch_size,1,tagset_size)

terminal_var = forward_var + transitions[tag2idx[STOP_TAG]]
alpha = torch.logsumexp(terminal_var,dim=-1)

alpha

tensor([[34.5524],
        [34.4737],
        [34.1782],
        [34.4694],
        [33.6329]])

In [125]:
a = torch.rand((3,3))
b = torch.zeros((5,5),dtype=torch.long).random_(0,3)

a,b,b[:,0],b[:,2]

a, b[:,0], b[:,2], a[b[:,0],b[:,2]].shape

(tensor([[0.8229, 0.9243, 0.0072],
         [0.9102, 0.1859, 0.4592],
         [0.9564, 0.7710, 0.3414]]),
 tensor([2, 0, 1, 0, 0]),
 tensor([2, 0, 0, 1, 2]),
 torch.Size([5]))

In [133]:
a = torch.rand((5,3))
a, b[:,1], a.gather(dim=-1,index=b[:,1].unsqueeze(dim=-1))

(tensor([[0.5340, 0.3721, 0.4909],
         [0.8295, 0.7753, 0.2160],
         [0.2526, 0.1448, 0.0755],
         [0.8207, 0.1910, 0.5065],
         [0.4993, 0.5855, 0.4707]]),
 tensor([2, 0, 1, 1, 1]),
 tensor([[0.4909],
         [0.8295],
         [0.1448],
         [0.1910],
         [0.5855]]))

In [136]:
a = torch.rand((5,5))
b = torch.zeros((2,5),dtype=torch.long).random_(0,5)

a, b[:,-1], a[0,b[:,-1]]

(tensor([[0.9712, 0.4389, 0.0791, 0.8923, 0.6217],
         [0.5868, 0.5858, 0.8488, 0.2723, 0.8644],
         [0.9808, 0.3711, 0.8156, 0.7759, 0.9682],
         [0.3836, 0.2026, 0.3144, 0.1598, 0.5172],
         [0.9821, 0.9851, 0.8777, 0.5495, 0.5109]]),
 tensor([4, 3]),
 tensor([0.6217, 0.8923]))

In [123]:
a = torch.tensor([[1,2,3],[2,3,4],[4,5,6]])
b = torch.tensor([[0,1],[1,0],[2,2]])
a.gather(dim=-1, index=b)

tensor([[1, 2],
        [3, 2],
        [6, 6]])

In [ ]:
transitions = torch.rand((10,10)

In [138]:
torch.manual_seed(1)

tagset_size=10
START_TAG=0
STOP_TAG=9
tag2idx = {k:k for k in range(10)}
transitions = torch.rand((10,10))

feats = torch.rand((batch_size, 10,10))

backpointers = []
# Initialize the viterbi variables in log space
init_vvars = torch.full((batch_size, 1, tagset_size), -10000.)
init_vvars[:, 0, tag2idx[START_TAG]] = 0.

# forward_var at step i holds the viterbi variables for step i-1
forward_var = init_vvars    # [1,tag_size]
for i in range(feats.shape[1]):
    feat = feats[:,i,:]

    # [tag_size]
    bptrs_t = []  # holds the backpointers for this step
    viterbivars_t = []  # holds the viterbi variables for this step
    next_tag_var = forward_var + transitions + feat.view(batch_size, tagset_size, 1)
    best_tag_var, best_tag_id = torch.max(next_tag_var, dim=-1)
    backpointers.append(best_tag_id)
    forward_var = best_tag_var.view(batch_size, 1, tagset_size)

    # for next_tag in range(tagset_size):
    #     # next_tag_var[i] holds the viterbi variable for tag i at the
    #     # previous step, plus the score of transitioning
    #     # from tag i to next_tag.
    #     # We don't include the emission scores here because the max
    #     # does not depend on them (we add them in below)
    #     next_tag_var = forward_var + transitions[next_tag] # [1,tag_size]
    #     best_tag_id = torch.argmax(next_tag_var, dim=-1)
    #     bptrs_t.append(best_tag_id) # best pre-tag of each next-tag
    #     viterbivars_t.append(next_tag_var[0][best_tag_id].view(1)) # value of variable
    # # Now add in the emission scores, and assign forward_var to the set
    # # of viterbi variables we just computed
    # forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1) # [1,tag_size]
    # backpointers.append(bptrs_t)

# Transition to STOP_TAG
terminal_var = forward_var + transitions[tag2idx[STOP_TAG]]
path_score, best_tag_id = torch.max(terminal_var, dim=-1)
print(terminal_var.shape, path_score.shape)
best_path = [best_tag_id]

for bptrs_t in reversed(backpointers):
    best_tag_id = bptrs_t.gather(dim=-1,index=best_tag_id)
    best_path.append(best_tag_id)
# Pop off the start tag (we dont want to return that to the caller)
start = best_path.pop()
best_path.reverse()
path_score, best_path

torch.Size([5, 1, 10]) torch.Size([5, 1])


(tensor([[17.6795],
         [17.1391],
         [17.3876],
         [16.9612],
         [16.0013]]),
 [tensor([[0],
          [4],
          [7],
          [4],
          [1]]),
  tensor([[7],
          [5],
          [7],
          [5],
          [3]]),
  tensor([[2],
          [7],
          [7],
          [5],
          [6]]),
  tensor([[4],
          [2],
          [2],
          [4],
          [9]]),
  tensor([[8],
          [3],
          [3],
          [5],
          [7]]),
  tensor([[1],
          [6],
          [6],
          [4],
          [2]]),
  tensor([[3],
          [3],
          [2],
          [5],
          [4]]),
  tensor([[4],
          [0],
          [7],
          [5],
          [8]]),
  tensor([[5],
          [7],
          [2],
          [7],
          [3]]),
  tensor([[0],
          [2],
          [6],
          [2],
          [6]])])

In [70]:
a = torch.full((5, 5, 1), -1.)
b = torch.rand((1,5,5))
a,b,a+b

(tensor([[[-1.],
          [-1.],
          [-1.],
          [-1.],
          [-1.]],
 
         [[-1.],
          [-1.],
          [-1.],
          [-1.],
          [-1.]],
 
         [[-1.],
          [-1.],
          [-1.],
          [-1.],
          [-1.]],
 
         [[-1.],
          [-1.],
          [-1.],
          [-1.],
          [-1.]],
 
         [[-1.],
          [-1.],
          [-1.],
          [-1.],
          [-1.]]]),
 tensor([[[0.7718, 0.3404, 0.1011, 0.5803, 0.4554],
          [0.4495, 0.2544, 0.5104, 0.6436, 0.6804],
          [0.0972, 0.7416, 0.5514, 0.8449, 0.8534],
          [0.1062, 0.9802, 0.0083, 0.7874, 0.5352],
          [0.7366, 0.2296, 0.8006, 0.2526, 0.0581]]]),
 tensor([[[-0.2282, -0.6596, -0.8989, -0.4197, -0.5446],
          [-0.5505, -0.7456, -0.4896, -0.3564, -0.3196],
          [-0.9028, -0.2584, -0.4486, -0.1551, -0.1466],
          [-0.8938, -0.0198, -0.9917, -0.2126, -0.4648],
          [-0.2634, -0.7704, -0.1994, -0.7474, -0.9419]],
 
         

In [66]:
init_alphas.shape

torch.Size([5, 1, 10])